### Importing Packages

In [1]:
import os
import pandas as pd 
import numpy as np 
import seaborn as sns 

### Importing Data

##### Importing train data

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/'):
    for filename in filenames:
        print(filename)

train_credit_bureau_a_1_3.csv
train_static_cb_0.csv
train_applprev_1_0.csv
train_person_2.csv
train_base.csv
train_tax_registry_a_1.csv
train_static_0_0.csv
train_credit_bureau_a_1_0.csv
train_applprev_2.csv
train_credit_bureau_a_2_6.csv
train_credit_bureau_a_1_2.csv
train_person_1.csv
train_credit_bureau_a_1_1.csv
train_tax_registry_c_1.csv
train_credit_bureau_a_2_4.csv
train_credit_bureau_a_2_9.csv
train_credit_bureau_a_2_3.csv
train_credit_bureau_a_2_7.csv
train_credit_bureau_b_2.csv
train_credit_bureau_a_2_2.csv
train_static_0_1.csv
train_deposit_1.csv
train_credit_bureau_a_2_10.csv
train_tax_registry_b_1.csv
train_applprev_1_1.csv
train_credit_bureau_a_2_1.csv
train_credit_bureau_a_2_8.csv
train_credit_bureau_a_2_5.csv
train_credit_bureau_b_1.csv
train_credit_bureau_a_2_0.csv
train_other_1.csv
train_debitcard_1.csv


In [3]:
filenames

['train_credit_bureau_a_1_3.csv',
 'train_static_cb_0.csv',
 'train_applprev_1_0.csv',
 'train_person_2.csv',
 'train_base.csv',
 'train_tax_registry_a_1.csv',
 'train_static_0_0.csv',
 'train_credit_bureau_a_1_0.csv',
 'train_applprev_2.csv',
 'train_credit_bureau_a_2_6.csv',
 'train_credit_bureau_a_1_2.csv',
 'train_person_1.csv',
 'train_credit_bureau_a_1_1.csv',
 'train_tax_registry_c_1.csv',
 'train_credit_bureau_a_2_4.csv',
 'train_credit_bureau_a_2_9.csv',
 'train_credit_bureau_a_2_3.csv',
 'train_credit_bureau_a_2_7.csv',
 'train_credit_bureau_b_2.csv',
 'train_credit_bureau_a_2_2.csv',
 'train_static_0_1.csv',
 'train_deposit_1.csv',
 'train_credit_bureau_a_2_10.csv',
 'train_tax_registry_b_1.csv',
 'train_applprev_1_1.csv',
 'train_credit_bureau_a_2_1.csv',
 'train_credit_bureau_a_2_8.csv',
 'train_credit_bureau_a_2_5.csv',
 'train_credit_bureau_b_1.csv',
 'train_credit_bureau_a_2_0.csv',
 'train_other_1.csv',
 'train_debitcard_1.csv']

In [4]:
df_base = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_base.csv")

In [5]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1526659 entries, 0 to 1526658
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   case_id        1526659 non-null  int64 
 1   date_decision  1526659 non-null  object
 2   MONTH          1526659 non-null  int64 
 3   WEEK_NUM       1526659 non-null  int64 
 4   target         1526659 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 58.2+ MB


In [6]:
df_base['case_id'].nunique()

1526659

In [7]:
df_static_0 = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_static_0_0.csv")
df_static_1 = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_static_0_1.csv")

/tmp/ipykernel_18/3297399063.py:1: DtypeWarning: Columns (20,45,46,53,57,84,143,146,167) have mixed types. Specify dtype option on import or set low_memory=False.
  df_static_0 = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_static_0_0.csv")
/tmp/ipykernel_18/3297399063.py:2: DtypeWarning: Columns (20,45,46,56,57,84,143,146,167) have mixed types. Specify dtype option on import or set low_memory=False.
  df_static_1 = pd.read_csv("/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_static_0_1.csv")


In [8]:
df_static_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003757 entries, 0 to 1003756
Columns: 168 entries, case_id to validfrom_1069D
dtypes: bool(1), float64(128), int64(1), object(38)
memory usage: 1.2+ GB


In [9]:
df_static_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522902 entries, 0 to 522901
Columns: 168 entries, case_id to validfrom_1069D
dtypes: bool(1), float64(128), int64(1), object(38)
memory usage: 666.7+ MB
